In [1]:
import pandas as pd
import re
from difflib import SequenceMatcher
from collections import defaultdict

In [2]:
## Path for restaurant data
restaurant_names = ["Gather24-25", "OK24-25", "Totem24-25"]
restaurant_data_paths = [f"Labelled_Data_{restaurant_name}_for_nutrislice.csv" for restaurant_name in restaurant_names]

In [3]:
results = []
for restaurant_name, restaurant_data_path in zip(restaurant_names, restaurant_data_paths):
    df = pd.read_csv(restaurant_data_path)
    df["restaurant"] = restaurant_name
    results.append(df)

results = pd.concat(results)
results.drop_duplicates(inplace=True)
results.reset_index(drop=True, inplace=True)
results

,Optimum Control ID,OC Description,SalesGroup,Combined Label,Red,Yellow,Green,GHG Emission (g),N lost (g),Land Use (m^2),Freshwater Withdrawals (L),Stress-Weighted Water Use (L),GHG Emission (g) / 100g,N lost (g) / 100g,Freshwater Withdrawals (L) / 100g,Stress-Weighted Water Use (L) / 100g,Land Use (m^2) / 100g,restaurant
0,R-73669,DEL|smoothie| mango pineapple,GV - DELISH,Green,False,False,True,77.151358,0.375085,0.148062,0.59,1.92,22.04,0.11,0.17,0.55,0.04,Gather24-25
1,R-56613,DEL|Smoothie|Almonds forever,GV - DELISH,Yellow,False,True,False,324.658690,1.122366,0.675339,101.87,7213.85,94.72,0.33,29.72,2104.70,0.20,Gather24-25
2,R-44306,DEL|Smoothie|Banana Matcha,GV - DELISH,Green,False,False,True,284.936566,1.539948,0.554830,71.85,4270.05,78.06,0.42,19.68,1169.88,0.15,Gather24-25
3,R-43697,DEL|Smoothie|Blueberry Boost,GV - DELISH,Red,True,False,False,1649.829015,6.896554,1.263809,281.74,11794.01,452.01,1.89,77.19,3231.24,0.35,Gather24-25
4,R-46074,DEL|Smoothie|Bna Almnd Buta,GV - DELISH,Green,False,False,True,169.432593,0.488466,0.549266,2.28,128.70,46.42,0.13,0.62,35.26,0.15,Gather24-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,P-26062,SALSA|Pico de Gallo,PREP,Green,False,False,True,21005.354382,257.200874,11.194883,2088.44,118743.83,70.02,0.86,6.96,395.81,0.04,Totem24-25
1152,P-33438,SAUCE|Hummus,PREP,Green,False,False,True,18485.853000,59.238300,108.363310,1946.67,137192.99,123.24,0.39,12.98,914.62,0.72,Totem24-25
1153,P-35848,SAUCE|Tzatziki Sauce,PREP,Yellow,False,True,False,6659.783239,61.316278,6.240908,618.01,29369.76,221.99,2.04,20.60,978.99,0.21,Totem24-25
1154,I-1033,BREAD PITA GREEK THK 7IN TFC,BAKED GOODS,Green,False,False,True,0.129412,1.258000,0.000416,0.04,1.09,0.15,1.48,0.05,1.28,0.00,Totem24-25


In [4]:
def analyze_recipes(df, column_name='OC Description'):
    """
    Analyze recipes and categorize them as animal-based or plant-based.
    Plant-based recipes must not contain any animal-based ingredients.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing recipe information
    column_name (str): Name of the column containing recipe descriptions
    
    Returns:
    tuple: (animal_recipes DataFrame, plant_recipes DataFrame, stats dictionary)
    """
    # Define comprehensive ingredient lists
    animal_based = [
        'beef', 'chicken', 'lamb', 'pork', 'fish', 'egg', 'salmon',
        'tuna', 'shrimp', 'turkey', 'meat', 'duck', 'bacon',
        'sausage', 'seafood', 'shellfish', "omelette", 'omelet'
    ]
    
    plant_based = [
        'tofu', 'lentil', 'vegan', 'plant', 'miso', 'tempeh',
        'seitan', 'chickpea', 'bean', 'quinoa', 'vegetarian',
        'soy', 'mushroom', 'veggie', 'falafel', 'hummus',
        'jackfruit', 'beyond', 'impossible'
    ]
    
    try:
        # Ensure the column exists
        if column_name not in df.columns:
            raise ValueError(f"Column '{column_name}' not found in DataFrame")
            
        # Convert column to string type and fill NaN values
        df[column_name] = df[column_name].astype(str).fillna('')
        
        # Create regex patterns with word boundaries
        animal_pattern = r'\b(' + '|'.join(animal_based) + r')\b'
        plant_pattern = r'\b(' + '|'.join(plant_based) + r')\b'
        
        # Filter recipes
        animal_recipes = df[df[column_name].str.contains(
            animal_pattern, 
            case=False, 
            regex=True
        )].copy()
        
        # For plant-based recipes:
        # 1. Must contain plant-based keywords
        # 2. Must NOT contain animal-based keywords
        plant_recipes = df[
            df[column_name].str.contains(
                plant_pattern,
                case=False,
                regex=True
            ) &
            ~df[column_name].str.contains(
                animal_pattern,
                case=False,
                regex=True
            )
        ].copy()
        
        # Add category labels
        animal_recipes['category'] = 'animal_based'
        plant_recipes['category'] = 'plant_based'
        
        # Calculate statistics
        stats = {
            'total_recipes': len(df),
            'animal_based_count': len(animal_recipes),
            'strict_plant_based_count': len(plant_recipes),
            'animal_based_percentage': round(len(animal_recipes) / len(df) * 100, 2),
            'strict_plant_based_percentage': round(len(plant_recipes) / len(df) * 100, 2),
            'uncategorized_count': len(df) - len(animal_recipes) - len(plant_recipes),
            'uncategorized_percentage': round((len(df) - len(animal_recipes) - len(plant_recipes)) / len(df) * 100, 2)
        }
        
        # Add matched keywords columns
        def find_matched_keywords(text, keywords):
            matches = [word for word in keywords if re.search(r'\b' + re.escape(word) + r'\b', text, re.IGNORECASE)]
            return ', '.join(matches) if matches else ''
        
        animal_recipes['matched_ingredients'] = animal_recipes[column_name].apply(
            lambda x: find_matched_keywords(x, animal_based)
        )
        plant_recipes['matched_ingredients'] = plant_recipes[column_name].apply(
            lambda x: find_matched_keywords(x, plant_based)
        )
        
        return animal_recipes, plant_recipes, stats
        
    except Exception as e:
        print(f"Error analyzing recipes: {str(e)}")
        return pd.DataFrame(), pd.DataFrame(), {}

In [5]:
animal_recipes, plant_recipes, stats = analyze_recipes(results)

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_5428/992152671.py:40: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  animal_recipes = df[df[column_name].str.contains(
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_5428/992152671.py:50: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[column_name].str.contains(
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_5428/992152671.py:55: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ~df[column_name].str.contains(


In [6]:
animal_recipes

,Optimum Control ID,OC Description,SalesGroup,Combined Label,Red,Yellow,Green,GHG Emission (g),N lost (g),Land Use (m^2),Freshwater Withdrawals (L),Stress-Weighted Water Use (L),GHG Emission (g) / 100g,N lost (g) / 100g,Freshwater Withdrawals (L) / 100g,Stress-Weighted Water Use (L) / 100g,Land Use (m^2) / 100g,restaurant,category,matched_ingredients
16,R-14085,DIM SUM|Bun|BBQ Pork (1),GV - DIM SUM,Red,True,False,False,482.545000,6.273000,0.000000,94.75,2850.24,567.70,7.38,111.47,3353.22,0.00,Gather24-25,animal_based,pork
17,R-37741,DIM SUM|Bun|Chicken (1),GV - DIM SUM,Green,False,False,True,288.096875,5.593375,0.000000,46.76,1048.22,198.69,3.86,32.25,722.91,0.00,Gather24-25,animal_based,chicken
34,R-65761,FF|Bowl|Poached Egg Zuc.|AAD,GV FRESH FARE DAY,Green,False,False,True,644.382248,4.382055,1.217309,184.00,1720.02,179.00,1.22,51.11,477.78,0.34,Gather24-25,animal_based,egg
43,R-70058,FF|Omelette|3Cheese|AAD,GV FRESH FARE BREAKFAST,Red,True,False,False,1573.110977,20.086341,2.389152,267.90,11401.72,555.87,7.10,94.66,4028.88,0.84,Gather24-25,animal_based,omelette
44,R-70095,FF|Omelette|Mushroom&Ched|AAD,GV FRESH FARE BREAKFAST,Red,True,False,False,967.823941,13.822301,1.728308,160.67,5805.33,406.65,5.81,67.51,2439.21,0.73,Gather24-25,animal_based,omelette
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,R-61979,HS|Tuscan Chicken|PLATE,FT HOMESKILLET,Yellow,False,True,False,1133.996739,21.186884,2.255985,114.65,3643.17,331.58,6.19,33.52,1065.25,0.66,Totem24-25,animal_based,chicken
1105,R-55967,RTS|Egg|Ajitama Ramen,FEAST,Red,True,False,False,561.196213,9.272708,0.999513,90.74,2661.74,1122.39,18.55,181.48,5323.48,2.00,Totem24-25,animal_based,egg
1125,R-73508,SOUP|Goodly|Beef Barley Shroom,FEAST,Red,True,False,False,12403.890000,98.850000,37.935000,503.16,18392.70,4134.63,32.95,167.72,6130.90,12.64,Totem24-25,animal_based,beef
1126,R-73213,SOUP|Goodly|Chicken Orzo,FEAST,Red,True,False,False,1319.880000,35.040000,3.453000,111.09,100.05,439.96,11.68,37.03,33.35,1.15,Totem24-25,animal_based,chicken


In [7]:
def find_similar_recipes(animal_recipes, plant_recipes, similarity_threshold=0.6):
    """
    Find similar recipe pairs between animal-based and plant-based categories.
    
    Parameters:
    animal_recipes (pandas.DataFrame): DataFrame of animal-based recipes
    plant_recipes (pandas.DataFrame): DataFrame of plant-based recipes
    similarity_threshold (float): Minimum similarity score to consider recipes as similar
    
    Returns:
    pandas.DataFrame: DataFrame containing similar recipe pairs and their dish types
    """
    
    def extract_dish_type(description):
        """Extract the basic dish type from a recipe description."""
        # Common dish types to look for
        dish_types = {
            'taco': ['taco', 'tacos'],
            'curry': ['curry', 'curries'],
            'burger': ['burger', 'burgers'],
            'soup': ['soup', 'soups', 'stew', 'stews'],
            'pasta': ['pasta', 'noodle', 'noodles', 'spaghetti', 'fettuccine'],
            'sandwich': ['sandwich', 'sandwiches'],
            'bowl': ['bowl', 'bowls'],
            'stir fry': ['stir fry', 'stir-fry'],
            'salad': ['salad', 'salads'],
            'wrap': ['wrap', 'wraps'],
            'rice': ['rice', 'risotto'],
            'pizza': ['pizza', 'pizzas'],
            'casserole': ['casserole', 'bake', 'hotdish']
        }
        
        description = description.lower()
        for dish_type, variants in dish_types.items():
            for variant in variants:
                if re.search(r'\b' + re.escape(variant) + r'\b', description):
                    return dish_type
        return 'other'

    def calculate_similarity(desc1, desc2):
        """Calculate similarity between two recipe descriptions."""
        # Remove common words that might interfere with similarity calculation
        common_words = {'and', 'with', 'the', 'a', 'an', 'in', 'on', 'at', 'to', 'for', 'of'}
        desc1 = ' '.join([word for word in desc1.lower().split() if word not in common_words])
        desc2 = ' '.join([word for word in desc2.lower().split() if word not in common_words])
        return SequenceMatcher(None, desc1, desc2).ratio()

    try:
        # Extract dish types for both categories
        animal_recipes['dish_type'] = animal_recipes['OC Description'].apply(extract_dish_type)
        plant_recipes['dish_type'] = plant_recipes['OC Description'].apply(extract_dish_type)
        
        # Initialize lists to store similar pairs
        similar_pairs = []
        
        # Group recipes by dish type
        animal_by_type = defaultdict(list)
        plant_by_type = defaultdict(list)
        
        for _, row in animal_recipes.iterrows():
            animal_by_type[row['dish_type']].append(row)
        for _, row in plant_recipes.iterrows():
            plant_by_type[row['dish_type']].append(row)
        
        # Find similar pairs within each dish type
        for dish_type in set(animal_by_type.keys()) & set(plant_by_type.keys()):
            for animal_recipe in animal_by_type[dish_type]:
                for plant_recipe in plant_by_type[dish_type]:
                    similarity = calculate_similarity(
                        animal_recipe['OC Description'],
                        plant_recipe['OC Description']
                    )
                    
                    if similarity >= similarity_threshold:
                        similar_pairs.append({
                            'dish_type': dish_type,
                            'animal_recipe': animal_recipe['OC Description'],
                            'animal_recipe_ID': animal_recipe['Optimum Control ID'],
                            'animal_ingredients': animal_recipe.get('matched_ingredients', ''),
                            'plant_recipe': plant_recipe['OC Description'],
                            'plant_recipe_ID': plant_recipe['Optimum Control ID'],
                            'plant_ingredients': plant_recipe.get('matched_ingredients', ''),
                            'similarity_score': round(similarity, 3)
                        })
        
        # Create DataFrame from similar pairs
        similar_df = pd.DataFrame(similar_pairs)
        
        # Sort by dish type and similarity score
        if not similar_df.empty:
            similar_df = similar_df.sort_values(
                ['dish_type', 'similarity_score'], 
                ascending=[True, False]
            )
            
            # Add summary statistics
            stats = {
                'total_similar_pairs': len(similar_df),
                'unique_dish_types': len(similar_df['dish_type'].unique()),
                'avg_similarity_score': round(similar_df['similarity_score'].mean(), 3),
                'dish_type_counts': similar_df['dish_type'].value_counts().to_dict()
            }
        else:
            stats = {
                'total_similar_pairs': 0,
                'unique_dish_types': 0,
                'avg_similarity_score': 0,
                'dish_type_counts': {}
            }
            
        return similar_df, stats

    except Exception as e:
        print(f"Error finding similar recipes: {str(e)}")
        return pd.DataFrame(), {}

In [8]:
similar_recipes, stats = find_similar_recipes(animal_recipes, plant_recipes, 0.7)

In [9]:
similar_recipes

,dish_type,animal_recipe,animal_recipe_ID,animal_ingredients,plant_recipe,plant_recipe_ID,plant_ingredients,similarity_score
29,bowl,HS|Bowl|Coconut Miso Salmon,R-73323,salmon,HS|Bowl|Coconut Miso Tofu,R-73499,"tofu, miso",0.846
28,bowl,CLV|Chicken Tikka Masala|BOWL,R-68226,chicken,CLV|Tofu Tikka Masala|BOWL,R-68229,tofu,0.800
2,burger,GRL|Burger|swiss chz|Pork|AAD,R-50118,pork,GRL|Burger|Swiss Mushroom|AAD,R-70492,mushroom,0.793
1,burger,GRL|Burger|Salmon|AAD,R-44931,salmon,GRL|Burger|Swiss Mushroom|AAD,R-70492,mushroom,0.720
0,burger,GRL|Burger|Salmon|AAD,R-44931,salmon,GRL|Burger|Jackfruit|AAD,R-53579,jackfruit,0.711
3,burger,GRL|Chicken Burger|Maui|AAD,R-56454,chicken,GRL|Burger|Jackfruit|AAD,R-53579,jackfruit,0.706
27,curry,SQR|Thai Green Fish Curry+1,R-57350,fish,SQR|TOFU|Thai Green Curry,R-49260,tofu,0.769
18,other,ALF|Flatbread|Shrimp Pesto,R-61742,shrimp,ALF|Flatbread|Mushroom Pesto,R-61779,mushroom,0.889
17,other,HC|Lemon Olive Oil Salmon|AAD,R-62715,salmon,HC|Lemon Olive Oil Tofu|AAD,R-73542,tofu,0.857
19,other,SQR|Citrus Soy Pork Skewer +1,R-62631,pork,SQR|Citrus Mushroom Skewer +1,R-62639,mushroom,0.828


In [10]:
visualisations = similar_recipes[["animal_recipe", "animal_recipe_ID", "plant_recipe", "plant_recipe_ID"]]
visualisations

,animal_recipe,animal_recipe_ID,plant_recipe,plant_recipe_ID
29,HS|Bowl|Coconut Miso Salmon,R-73323,HS|Bowl|Coconut Miso Tofu,R-73499
28,CLV|Chicken Tikka Masala|BOWL,R-68226,CLV|Tofu Tikka Masala|BOWL,R-68229
2,GRL|Burger|swiss chz|Pork|AAD,R-50118,GRL|Burger|Swiss Mushroom|AAD,R-70492
1,GRL|Burger|Salmon|AAD,R-44931,GRL|Burger|Swiss Mushroom|AAD,R-70492
0,GRL|Burger|Salmon|AAD,R-44931,GRL|Burger|Jackfruit|AAD,R-53579
3,GRL|Chicken Burger|Maui|AAD,R-56454,GRL|Burger|Jackfruit|AAD,R-53579
27,SQR|Thai Green Fish Curry+1,R-57350,SQR|TOFU|Thai Green Curry,R-49260
18,ALF|Flatbread|Shrimp Pesto,R-61742,ALF|Flatbread|Mushroom Pesto,R-61779
17,HC|Lemon Olive Oil Salmon|AAD,R-62715,HC|Lemon Olive Oil Tofu|AAD,R-73542
19,SQR|Citrus Soy Pork Skewer +1,R-62631,SQR|Citrus Mushroom Skewer +1,R-62639


In [11]:
# Add new columns for each emission metric in animal and plant-based recipes
for metric in ["GHG Emission (g) / 100g", "N lost (g) / 100g", "Freshwater Withdrawals (L) / 100g", 
               "Stress-Weighted Water Use (L) / 100g", "Land Use (m^2) / 100g"]:
    
    # Extract emissions for animal-based recipes
    visualisations[f"animal_{metric}"] = visualisations['animal_recipe_ID'].apply(
        lambda recipe_id: results.loc[results['Optimum Control ID'] == recipe_id, metric].values[0]
    )
    
    # Extract emissions for plant-based recipes
    visualisations[f"plant_{metric}"] = visualisations['plant_recipe_ID'].apply(
        lambda recipe_id: results.loc[results['Optimum Control ID'] == recipe_id, metric].values[0]
    )

# Show the updated DataFrame
visualisations.head()


/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_5428/185445706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visualisations[f"animal_{metric}"] = visualisations['animal_recipe_ID'].apply(
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_5428/185445706.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visualisations[f"plant_{metric}"] = visualisations['plant_recipe_ID'].apply(
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_5428/185445706.py:6: SettingWithCopyWarning:

,animal_recipe,animal_recipe_ID,plant_recipe,plant_recipe_ID,animal_GHG Emission (g) / 100g,plant_GHG Emission (g) / 100g,animal_N lost (g) / 100g,plant_N lost (g) / 100g,animal_Freshwater Withdrawals (L) / 100g,plant_Freshwater Withdrawals (L) / 100g,animal_Stress-Weighted Water Use (L) / 100g,plant_Stress-Weighted Water Use (L) / 100g,animal_Land Use (m^2) / 100g,plant_Land Use (m^2) / 100g
29,HS|Bowl|Coconut Miso Salmon,R-73323,HS|Bowl|Coconut Miso Tofu,R-73499,328.88,147.49,3.50,0.58,92.00,23.09,542.99,163.86,0.47,0.27
28,CLV|Chicken Tikka Masala|BOWL,R-68226,CLV|Tofu Tikka Masala|BOWL,R-68229,417.22,204.23,5.08,1.07,76.63,38.63,1785.27,595.57,0.87,0.36
2,GRL|Burger|swiss chz|Pork|AAD,R-50118,GRL|Burger|Swiss Mushroom|AAD,R-70492,550.19,1523.06,6.96,12.30,104.38,78.11,3478.62,2925.84,1.15,4.65
1,GRL|Burger|Salmon|AAD,R-44931,GRL|Burger|Swiss Mushroom|AAD,R-70492,349.97,1523.06,4.24,12.30,95.86,78.11,1028.02,2925.84,0.70,4.65
0,GRL|Burger|Salmon|AAD,R-44931,GRL|Burger|Jackfruit|AAD,R-53579,349.97,131.20,4.24,0.86,95.86,14.18,1028.02,434.40,0.70,0.30


In [12]:
# Filter out rows where animal GHG emissions are less than plant GHG emissions
def filter_emissions(row):
    """
    Filter out rows where animal emissions are less than plant emissions.

    If more than one emission metric is lower for animal-based recipes compared to plant-based recipes,
    the row will be filtered out.

    Parameters:
    row (pandas.Series): Row of the DataFrame

    Returns:
    bool: True if animal emissions are greater than or equal to plant emissions, False otherwise
    """
    # Extract emission columns
    animal_emissions = [row[f'animal_{metric}'] for metric in ["GHG Emission (g) / 100g", "N lost (g) / 100g", 
                                                                "Freshwater Withdrawals (L) / 100g", 
                                                                "Stress-Weighted Water Use (L) / 100g", 
                                                                "Land Use (m^2) / 100g"]]
    plant_emissions = [row[f'plant_{metric}'] for metric in ["GHG Emission (g) / 100g", "N lost (g) / 100g", 
                                                              "Freshwater Withdrawals (L) / 100g", 
                                                              "Stress-Weighted Water Use (L) / 100g", 
                                                              "Land Use (m^2) / 100g"]]

    # Check if animal emissions are greater than or equal to plant emissions
    cnt = 1
    for animal, plant in zip(animal_emissions, plant_emissions):
        if animal < plant:
            cnt += 1
        if cnt > 2:
            return False
    return True
    
visualisations = visualisations[visualisations.apply(filter_emissions, axis=1)]

# Show the updated DataFrame
visualisations

,animal_recipe,animal_recipe_ID,plant_recipe,plant_recipe_ID,animal_GHG Emission (g) / 100g,plant_GHG Emission (g) / 100g,animal_N lost (g) / 100g,plant_N lost (g) / 100g,animal_Freshwater Withdrawals (L) / 100g,plant_Freshwater Withdrawals (L) / 100g,animal_Stress-Weighted Water Use (L) / 100g,plant_Stress-Weighted Water Use (L) / 100g,animal_Land Use (m^2) / 100g,plant_Land Use (m^2) / 100g
29,HS|Bowl|Coconut Miso Salmon,R-73323,HS|Bowl|Coconut Miso Tofu,R-73499,328.88,147.49,3.50,0.58,92.00,23.09,542.99,163.86,0.47,0.27
28,CLV|Chicken Tikka Masala|BOWL,R-68226,CLV|Tofu Tikka Masala|BOWL,R-68229,417.22,204.23,5.08,1.07,76.63,38.63,1785.27,595.57,0.87,0.36
0,GRL|Burger|Salmon|AAD,R-44931,GRL|Burger|Jackfruit|AAD,R-53579,349.97,131.20,4.24,0.86,95.86,14.18,1028.02,434.40,0.70,0.30
3,GRL|Chicken Burger|Maui|AAD,R-56454,GRL|Burger|Jackfruit|AAD,R-53579,350.96,131.20,6.46,0.86,37.24,14.18,898.00,434.40,0.94,0.30
27,SQR|Thai Green Fish Curry+1,R-57350,SQR|TOFU|Thai Green Curry,R-49260,338.62,230.13,3.00,0.40,83.59,0.91,404.84,21.85,0.60,0.22
17,HC|Lemon Olive Oil Salmon|AAD,R-62715,HC|Lemon Olive Oil Tofu|AAD,R-73542,266.89,134.80,3.03,0.64,95.22,38.02,520.67,213.54,0.41,0.27
19,SQR|Citrus Soy Pork Skewer +1,R-62631,SQR|Citrus Mushroom Skewer +1,R-62639,480.57,154.09,5.68,1.19,113.47,56.52,2842.20,1076.23,0.96,0.26
16,FLX|Spicy Soy Salmon 100g|AAD,R-61967,FLX|Spicy Soy tofu 150g|AAD,R-62288,624.10,158.27,8.47,0.70,186.12,2.34,1036.30,88.66,0.98,0.37
20,SQR|Spicy Soy Salmon+1,R-41085,SQR|Spicy Soy Tofu+1,R-41089,341.54,152.23,3.70,0.50,101.29,24.90,500.00,90.84,0.42,0.21
24,HS|Sesame Ginger Chicken|PLATE,R-73143,HS|Sesame Ginger Tofu|PLATE,R-62138,261.39,148.38,4.90,0.69,38.96,28.19,307.50,234.94,0.68,0.36


In [13]:
signage_OK = pd.read_excel("signage_OK.xlsx")
signage_OK.drop_duplicates(inplace=True)
signage_OK.dropna(subset=["Station Name"], inplace=True)
signage_OK = signage_OK[["Item Name", "Internal Name"]]
signage_OK.reset_index(drop=True, inplace=True)
signage_OK


/Users/vivaanwadhwa/Documents/GitHub/environmental-dashboard/env/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Item Name,Internal Name
0,AS YOU WOULD LIKE IT TO APPEAR ON NUTRISLICE,COPY AND PASTE FROM OC
1,Apple Cinnamon Crepe,GRL|Crepe|Apple Cinnamon
2,Apple Pie Overnight Oats,OATS|Over Night|Apple Pie
3,Asparagus Quiche,VEG|Quiche|Asparagus
4,Assorted Fruit Overnight Oats,OATS|Over Night|Base
...,...,...
427,Tropical Smoothie,SMOOTHIE|Tropical
428,Wild Berry Smoothie,SMOOTHIE|Wild Berry
429,Halloumi Avocado Bagel,GRL|Halloumi|Avocado|Bagel
430,Ham and Cheese Bagel,GRL|Ham|Cheese|Bagel


In [14]:
signage_gather = pd.read_excel("signage_gather.xlsx")
signage_gather.drop_duplicates(inplace=True)
signage_gather.dropna(subset=["Station Name"], inplace=True)
signage_gather = signage_gather[["Item Name", "Internal Name"]]
signage_gather.reset_index(drop=True, inplace=True)
signage_gather

/Users/vivaanwadhwa/Documents/GitHub/environmental-dashboard/env/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Item Name,Internal Name
0,AS YOU WOULD LIKE IT TO APPEAR ON NUTRISLICE,COPY AND PASTE FROM OC
1,Asparagus benny,GRL|Benny|Asparagus|AAD
2,Asparagus poached egg bowl,GRL|BOWL|Asparagus poached egg
3,Avocado spinach benny,GRL|Benny |Avocado spinach|AAD
4,B.E.L.T. sandwich,GRL|Sand|B.E.L.T.|AAD
...,...,...
458,Roasted parsnips,SIDE BAR|RATIO|AAD
459,Roasted pepper,SIDE BAR|RATIO|AAD
460,Roasted zucchini,SIDE BAR|RATIO|AAD
461,Steamed broccoli,SIDE BAR|RATIO|AAD


In [15]:
signage_feast = pd.read_excel("signage_feast.xlsx")
signage_feast.drop_duplicates(inplace=True)
signage_feast.dropna(subset=["Station Name"], inplace=True)
signage_feast = signage_feast[["Item Name", "Internal Name"]]
signage_feast.reset_index(drop=True, inplace=True)
signage_feast


/Users/vivaanwadhwa/Documents/GitHub/environmental-dashboard/env/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Item Name,Internal Name
0,AS YOU WOULD LIKE IT TO APPEAR ON NUTRISLICE,COPY AND PASTE FROM OC
1,APPLE CINNAMON PANCAKES,DNR|Pancake|Apple Cinnamon
2,BACON STRIPS,DNR|Add Bacon|2 pcs
3,BANANA CARAMEL PANCAKES,DNR|Pancake|BananaCaramel|2pc
4,BLUEBERRY PANCAKES,DNR|Pancakes|Blueberry|2PC
...,...,...
343,HEARTY TOMATO VEGETABLE,SOUP|Goodly|Hearty tomato
344,SPICY CORN CHOWDER,SOUP|Goodly|Corn Chowder
345,BEEF BARLEY MUSHROOM,SOUP|Goodly|Beef Barley Shroom
346,HOUSE MADE CREAM OF MUSHROOM SOUP,SOUP|House made|Cream Of Shroo


In [16]:
for idx,row in visualisations.iterrows():
    for _,row2 in signage_OK.iterrows():
        if row["animal_recipe"] == row2["Internal Name"]:
            visualisations.loc[idx, "animal_recipe"] = row2["Item Name"].upper()
        if row["plant_recipe"] == row2["Internal Name"]:
            visualisations.loc[idx, "plant_recipe"] = row2["Item Name"].upper()
    for _,row3 in signage_gather.iterrows():
        if row["animal_recipe"] == row3["Internal Name"]:
            visualisations.loc[idx, "animal_recipe"] = row3["Item Name"].upper()
        if row["plant_recipe"] == row3["Internal Name"]:
            visualisations.loc[idx, "plant_recipe"] = row3["Item Name"].upper()
    for _,row4 in signage_feast.iterrows():
        if row["animal_recipe"] == row4["Internal Name"]:
            visualisations.loc[idx, "animal_recipe"] = row4["Item Name"].upper()
        if row["plant_recipe"] == row4["Internal Name"]:
            visualisations.loc[idx, "plant_recipe"] = row4["Item Name"].upper()

In [17]:
visualisations

,animal_recipe,animal_recipe_ID,plant_recipe,plant_recipe_ID,animal_GHG Emission (g) / 100g,plant_GHG Emission (g) / 100g,animal_N lost (g) / 100g,plant_N lost (g) / 100g,animal_Freshwater Withdrawals (L) / 100g,plant_Freshwater Withdrawals (L) / 100g,animal_Stress-Weighted Water Use (L) / 100g,plant_Stress-Weighted Water Use (L) / 100g,animal_Land Use (m^2) / 100g,plant_Land Use (m^2) / 100g
29,COCONUT MISO SALMON BOWL,R-73323,COCONUT MISO TOFU BOWL,R-73499,328.88,147.49,3.50,0.58,92.00,23.09,542.99,163.86,0.47,0.27
28,CHICKEN TIKKA MASALA BOWL,R-68226,TOFU TIKKA MASALA BOWL,R-68229,417.22,204.23,5.08,1.07,76.63,38.63,1785.27,595.57,0.87,0.36
0,SALMON BURGER,R-44931,JACKFRUIT BURGER,R-53579,349.97,131.20,4.24,0.86,95.86,14.18,1028.02,434.40,0.70,0.30
3,MAUI CHICKEN BURGER,R-56454,JACKFRUIT BURGER,R-53579,350.96,131.20,6.46,0.86,37.24,14.18,898.00,434.40,0.94,0.30
27,SQR|Thai Green Fish Curry+1,R-57350,SQR|TOFU|Thai Green Curry,R-49260,338.62,230.13,3.00,0.40,83.59,0.91,404.84,21.85,0.60,0.22
17,SALMON LEMON OLIVE OIL,R-62715,HC|Lemon Olive Oil Tofu|AAD,R-73542,266.89,134.80,3.03,0.64,95.22,38.02,520.67,213.54,0.41,0.27
19,CITRUS PORK SKEWER,R-62631,CITRUS MUSHROOM SKEWER,R-62639,480.57,154.09,5.68,1.19,113.47,56.52,2842.20,1076.23,0.96,0.26
16,SPICY SOY SALMON,R-61967,SPICY SOY TOFU,R-62288,624.10,158.27,8.47,0.70,186.12,2.34,1036.30,88.66,0.98,0.37
20,SQR|Spicy Soy Salmon+1,R-41085,SQR|Spicy Soy Tofu+1,R-41089,341.54,152.23,3.70,0.50,101.29,24.90,500.00,90.84,0.42,0.21
24,SESAME GINGER CHICKEN PLATE,R-73143,SESAME GINGER TOFU PLATE,R-62138,261.39,148.38,4.90,0.69,38.96,28.19,307.50,234.94,0.68,0.36


In [18]:
visualisations.to_csv("data/similar_recipes.csv", index=False)

In [ ]:
# sns.set_theme(style="whitegrid")

# # Create a color palette for animal vs plant-based recipes
# colors = ['#FF6F61', '#88B04B']  # Coral for animal, green for plant

# # Function to create a bar chart for a given metric
# def plot_comparison(metric, ylabel, title):
#     plt.figure(figsize=(10, 6))
    
#     # Bar plot for the selected metric
#     sns.barplot(x='animal_recipe', y=f'animal_{metric}', data=visualisations, color=colors[0], label='Animal-based')
#     sns.barplot(x='animal_recipe', y=f'plant_{metric}', data=visualisations, color=colors[1], label='Plant-based')
    
#     # Set plot labels and title
#     plt.ylabel(ylabel)
#     plt.title(title)
#     plt.xticks(rotation=45, ha='right')
#     plt.legend(loc='upper right')
    
#     # Annotate each bar with the exact value
#     for idx, row in visualisations.iterrows():
#         animal_val = row[f'animal_{metric}']
#         plant_val = row[f'plant_{metric}']
#         plt.text(idx, animal_val + 10, f'{animal_val:.2f}', color='black', ha="center")
#         plt.text(idx, plant_val + 10, f'{plant_val:.2f}', color='black', ha="center")
    
#     plt.tight_layout()
#     plt.show()

# # Plot comparisons for each environmental metric
# plot_comparison("GHG Emission (g) / 100g", "GHG Emission (g) per 100g", "GHG Emissions: Animal-based vs Plant-based Recipes")
# plot_comparison("N lost (g) / 100g", "Nitrogen Lost (g) per 100g", "Nitrogen Lost: Animal-based vs Plant-based Recipes")
# plot_comparison("Freshwater Withdrawals (L) / 100g", "Freshwater Withdrawals (L) per 100g", "Freshwater Withdrawals: Animal-based vs Plant-based Recipes")
# plot_comparison("Stress-Weighted Water Use (L) / 100g", "Stress-Weighted Water Use (L) per 100g", "Stress-Weighted Water Use: Animal-based vs Plant-based Recipes")
# plot_comparison("Land Use (m^2) / 100g", "Land Use (m²) per 100g", "Land Use: Animal-based vs Plant-based Recipes")